In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import os
import torch
import wandb
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted index.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any script options
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=True)

--- Logging error ---
Traceback (most recent call last):
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/logging/__init__.py", line 1081, in emit
    msg = self.format(record)
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/logging/__init__.py", line 664, in format
    record.message = record.getMessage()
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/t

In [ ]:
cfg.wandb['mode'] = 'online'
trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, optimizer,
                  ds, initial_eval=False, use_cpu=False)
trainer.train()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find run.
wandb: Currently logged in as: uts_nlp (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Launching training on one GPU.


  0%|          | 0/6 [00:00<?, ?it/s]

travis_attack.trainer: INFO     Now on epoch 1 of 3


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

travis_attack.trainer: INFO     Now on epoch 2 of 3


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 376.00 MiB (GPU 0; 15.75 GiB total capacity; 14.02 GiB already allocated; 345.06 MiB free; 14.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
## TO RESUME RUN
# cfg = Config()
# cfg.run_id = '3g070m4d'
# cfg.run_name = "avid-darkness-324"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=2)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()